# Imports

In [ ]:
BASE_DIR = '/content/drive/MyDrive/BrainTumorClassification'
TRAIN_DIR = f'{BASE_DIR}/data/Training'
TEST_DIR = f'{BASE_DIR}/data/Testing'
MOBILENET_PATH = f'{BASE_DIR}/saved_models/mobilenet_model.keras'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/BrainTumorClassification')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.models import load_model


# MobileNet Model Building

In [ ]:
def build_mobilenet_model(img_size=(224, 224), num_classes=4):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# ResNet Model Building

In [ ]:
def build_resnet_model(img_size=(224, 224), num_classes=4):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


# EfficientNetB0 Model Building

In [ ]:
def build_effnet_model(img_size=(240, 240), num_classes=4):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


# Train Models

In [ ]:
def train_model(model, train_gen, val_gen, test_gen, epochs=10, class_weight=None):
    """
    Trains the model with early stopping and evaluates it on test data.

    Parameters:
        model: Compiled Keras model
        train_gen: Training data generator
        val_gen: Validation data generator
        test_gen: Test data generator
        epochs: Number of training epochs
        class_weight: Optional dictionary mapping class indices to weights

    Returns:
        model: Trained model
        history: Keras History object
    """
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    )

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stop],
        verbose=1,
        class_weight=class_weight  # 🧠 Only used if not None
    )

    # Evaluate on test set
    test_loss, test_acc = model.evaluate(test_gen, verbose=0)
    print(f"\n📊 Final Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")

    return model, history



# Fine Tune Models

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

def finetune_mobilenet_model(model_path, fine_tune_at=-30, learning_rate=1e-5):
    model = load_model(model_path)

    # Make all layers trainable
    model.trainable = True

    # Freeze all layers up to fine_tune_at
    for layer in model.layers[:fine_tune_at]:
        layer.trainable = False

    # Re-compile with a lower learning rate for fine-tuning
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

